In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
def brusselator_1D_system(Dx, Dy,L,A,B,z_L,N):
    """
    Discretizes the 1D Brusselator equation using finite differences with Dirichlet BCs.
    
    Parameters:
        Dx,Dy (float): Diffusion coefficient.
        z_L (float): Length of the domain [0, L].
        N (int): Number of grid points (mesh size).
    
    Returns:
        callable: A function that computes du/dt for use in an ODE integrator.
    """
    # Discretize space
    h = z_L / (N - 1)  # Grid spacing
    
    def dydt(t, y):
        """
        Computes the time derivative du/dt for the reaction-diffusion system.
        
        Parameters:
            t (float): Current time (for time-dependent problems).
            u (np.array): Current state vector (size N).
        
        Returns:
            np.array: du/dt (size 2*N).
        """
        X = y[:N]
        Y = y[N:] 

        # Finite difference approximation for the second derivative (d^2u/dx^2)
        d2Xdz2 = np.zeros_like(X)
        d2Ydz2 = np.zeros_like(Y)

        # Inner points (central difference)
        d2Xdz2[1:-1] = (X[:-2] - 2 * X[1:-1] + X[2:]) / h**2
        d2Ydz2[1:-1] = (Y[:-2] - 2 * Y[1:-1] + Y[2:]) / h**2

        
        
        # Reaction-diffusion equation
        dXdt = Dx/(L**2) * d2Xdz2 + Y*X**2 - (B+1)*X + A
        dYdt = Dy/(L**2) * d2Ydz2 - Y*X**2 + B*X

        #Enforce Dirichlet BCs
        dXdt[0] = dXdt[-1] = 0  # X(0, t) =X(Z_L, t) = A
        dYdt[-0] = dYdt[-1] = 0  #X(0, t)  = Y(z_L, t) = B/A

        dydt = np.concatenate([dXdt, dYdt])
        
        return dydt
    
    return dydt

def brusselator_jacobian(X, Y, Dx, Dy,L,A,B,z_L,N):
    """
    Computes the Jacobian matrix for the discretized Brusselator system.
    
    Parameters:
        X (np.array): Concentration of X at each grid point (size N).
        Y (np.array): Concentration of Y at each grid point (size N).
        Dx (float): Diffusion coefficient for X.
        Dy (float): Diffusion coefficient for Y.
        dz (float): Spatial step size.
        A (float): Parameter A.
        B (float): Parameter B.
    
    Returns:
        J (np.array): Jacobian matrix (size 2N x 2N).
    """
    h = z_L / (N - 1)
    J = np.zeros((2 * (N-2), 2 * (N-2)))  # Initialize Jacobian
    
    # Diffusion coefficients
    alpha_x = Dx / (L*h)**2
    alpha_y = Dy / (L*h)**2
    
    # Fill J_XX and J_YY (tridiagonal blocks)
    for i in range(N-2):
        # Diagonal entries
        J[i, i] = -2 * alpha_x - (B + 1) + 2 * X[i] * Y[i]  # J_XX
        J[N-2 + i, N-2 + i] = -2 * alpha_y - X[i]**2  # J_YY
        
        # Off-diagonal entries (diffusion terms)
        if i > 0:
            J[i, i - 1] = alpha_x  # J_XX
            J[N-2 + i, N-2 + i - 1] = alpha_y  # J_YY
        if i < N-2 - 1:
            J[i, i + 1] = alpha_x  # J_XX
            J[N-2 + i, N-2 + i + 1] = alpha_y  # J_YY
    
    # Fill J_XY and J_YX (diagonal blocks)
    for i in range(N-2):
        J[i, N-2 + i] = X[i]**2  # J_XY
        J[N-2 + i, i] = B - 2 * X[i] * Y[i]  # J_YX
    
    return J


In [ ]:
def brusselator(Dx, Dy,L,A,B,z_L,N):
    
    # Discretize space
    h = z_L / (N - 1)  # Grid spacing
    
    def dydt(t, y):
        X = Y = np.zeros((N))
        X[1:-1] = y[:N-2]
        Y[1:-1] = y[N-2:] 
        #Dirichlet BCs
        Y[0] = Y[-1] = B/A  
        X[0] = X[-1] = A 
        # Finite difference approximation for the second derivative (d^2u/dx^2)
        d2Xdz2 = np.zeros(N)
        d2Ydz2 = np.zeros(N)
        # Central difference for ALL INNER POINTS (indices 1 to N-2)
        for i in range(1, N-1):
            d2Xdz2[i] = (X[i-1] - 2 * X[i] + X[i+1]) / h**2
            d2Ydz2[i] = (Y[i-1] - 2 * Y[i] + Y[i+1]) / h**2
        # Reaction-diffusion equation
        dXdt = Dx/(L**2) * d2Xdz2[1:-1] + Y[1:-1]*X[1:-1]**2 - (B+1)*X[1:-1] + A
        dYdt = Dy/(L**2) * d2Ydz2[1:-1] - Y[1:-1]*X[1:-1]**2 + B*X[1:-1]

        dydt = np.concatenate([dXdt, dYdt])
        return dydt
    
    return dydt

In [ ]:
x = np.array([1,2,3])
x = 5 + 0*x
x

## Trial Run

In [ ]:
# Example usage
if __name__ == "__main__":
    # Parameters
    Dx = 0.008
    Dy = 0.004
    L = 0.5130 #characteristic length
    z_L = 10  #Domain length
    A,B = 2,5.45   
    N = 100 # Number of grid points
 
    
    # Initial condition (e.g., a Gaussian pulse)
    z = np.linspace(0, z_L, N)
    perturb = 0.1 * np.exp(-((z - z_L / 2)**2)) #*np.concatenate([A*np.ones(N),(B/A)*np.ones(N)])
    
    X0 = A + perturb
    Y0 = B/A + perturb
    # # Enforce Dirichlet BCs on the initial condition
    Y0[0] = Y0[-1] = B/A  # u(0, t) = 0
    X0[0] = X0[-1] = A  # u(L, t) = 0
    y0 = np.concatenate([X0[1:-1],Y0[1:-1]])
    print("y0 shape", y0.shape)
    # Create the system
    system = brusselator(Dx, Dy, L, A,B,z_L,N)
    # Solve using an ODE integrator (e.g., scipy.integrate.solve_ivp)
    
    t_span = (0, 100)  # Time span
    sol = solve_ivp(system, t_span, y0, method='BDF', t_eval=np.linspace(0, 100, 100))
    print(sol.y.shape)
    # Plot the solution
    
    # plt.figure()
    # for i, t in enumerate(sol.t):
    #     plt.plot(z, sol.y[N:2*N, i], label=f"t={t:.2f}")
    # plt.xlabel("z")
    # plt.ylabel("X(t,z)")
    # # plt.legend()
    # plt.title("1D Reaction-Diffusion Equation with Dirichlet BCs")
    # plt.show()

    

In [ ]:
# Plot the solution
plt.figure(figsize=(10, 6))
for i, t in enumerate(sol.t):
    if i % 40 == 0:  # Plot every 50th time step
        # plt.plot(z[1:-1], sol.y[:N-2, i],'+', label=f"X, t={t:.2f}")
        plt.plot(z[1:-1], sol.y[N-2:, i],'+', label=f"Y, t={t:.2f}")
plt.xlabel("z")
plt.ylabel("Concentration")
plt.legend()
plt.title("Brusselator Model with Dirichlet BCs")
plt.show()


In [ ]:
Xmean = sol.y[:N,:]
Xmean = np.mean(Xmean,axis = 0)
Ymean = sol.y[N:,:]
Ymean = np.mean(Ymean,axis = 0)
plt.plot(np.linspace(0, 100, 100), Xmean)
plt.show()
plt.plot(Xmean,Ymean)

In [ ]:
J = brusselator_jacobian(X0[1:-1], Y0[1:-1], Dx, Dy,L,A,B,z_L,N)
J